In [10]:
# import the necessary packages
import argparse
import cv2
import matplotlib.pyplot as plt
from os import walk
import os
import glob
import matplotlib
import pandas as pd

In [11]:
print(cv2.__version__)

4.9.0


In [12]:
pore_data = pd.DataFrame(columns=["img", "label", "x1", "y1", "x2", "y2"])
disposal_pore = pd.DataFrame(columns=["img", "label", "x1", "y1", "x2", "y2"])
pore_data

,img,label,x1,y1,x2,y2


In [13]:
def load_all_image_path(img_dir):
    #img_dir = "./Build2" # Enter Directory of all images
    img_labels = []
    for(_, _, filenames) in walk(img_dir):
        img_labels.extend(filenames)
        break
    data_path = os.path.join(img_dir,'*g')
    files = glob.glob(data_path)
    return files, img_labels

In [158]:
def isKeepable(x, y, w, h, pore_data):
    for _, p in pore_data.iterrows():
        distance = ((x - p.x1)**2 + (y - p.y1)**2)**(1/2)
        
        if distance < 5:
            pore_data
            return [False, p, distance]
            
    return [True]

In [159]:
pore_data = pd.DataFrame(columns=["img", "label", "x1", "y1", "x2", "y2"])
disposal_pore = pd.DataFrame(columns=["img", "label", "x1", "y1", "x2", "y2", "pore close with", "distance"])

def CCA_pore(img):
    ## load the input image from disk 
    image = cv2.imread(img)
    #image = cv2.imread("license_plat.png")

    ## convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    ## threshold image
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    ## apply connected component analysis to the thresholded image
    output = cv2.connectedComponentsWithStats(thresh,8, cv2.CV_32S)
    (numLabels, labels, stats, centroids) = output
    
    #### total detected instances
    #print("Total number of objects: ", numLabels)

    pore_output = image.copy()
    b_box = 0

    # loop over the number of unique connected component labels
    for i in range(0, numLabels):
        # extract the connected component statistics and centroid for the current label
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        (cX, cY) = centroids[i]

        ## If the pore area is bigger then 5 only than draw a bounding box
        if (area > 5 and area < 200):
#             print("Width: " + str(w) + " Height: " + str(h) + " Area: " + str(area))

            ## bounding box counter
            b_box+=1

            keep = isKeepable(x, y, pore_data[pore_data['img'] == img[-15:]])
            if keep[0]:
                pore_data.loc[pore_data.shape[0]] = [img[-15:], 0, x, y, x + w, y + h]
            else:
                disposal_pore.loc[disposal_pore.shape[0]] = [img[-15:], 1, x, y, x + w, y + h, keep[1].name, keep[2]]
                continue
                
            ## Draw bounding box and a circle for the center of bounding box around the pore
            pore_output = cv2.rectangle(pore_output, (x-20, y-20), (x + w+20, y + h+20), (0, 255, 0), 1)
            pore_output = cv2.circle(pore_output, (int(cX), int(cY)), 4, (0, 0, 255), 1)
            
    #print(b_box)
    #plt.imshow(pore_output)
    return pore_output, b_box

## Make directories to store labelled images

In [160]:
# Make directories to store crop images if the the directories don't exsists already
random_directories = ["Corrections/4 - Object detection/Labeled_CCA/Test/", 
                      "Corrections/4 - Object detection/Labeled_CCA/Train/",
                      "Corrections/4 - Object detection/Labeled_CCA/Valid/"]

for directory in random_directories:
    if not os.path.exists(directory):
        os.makedirs(directory)
       
### delete old files in the directories
for directory in random_directories:
    files = glob.glob(os.path.join(directory,"*"))
    for f in files:
        os.remove(f)

### Testing Images

In [163]:
keep = isKeepable(pore_data.iloc[0].x1, pore_data.iloc[0].y1, pore_data[pore_data["img"] == "A20X_7_0266.jpg"])
keep

IndexError: single positional indexer is out-of-bounds

In [168]:
disposal_pore.loc[0, :] = disposal_pore.iloc[1]

In [169]:
disposal_pore

,img,label,x1,y1,x2,y2,pore close with,distance
0,A20X_7_0267.jpg,1,579,322,583,325,55,4.472136
1,A20X_7_0267.jpg,1,579,322,583,325,55,4.472136
2,A20X_7_0268.jpg,1,891,507,895,510,109,4.472136
3,A20X_7_0268.jpg,1,1596,511,1600,514,110,4.000000
4,A20X_7_1160.jpg,1,602,807,605,810,334,4.472136
5,A20X_7_1160.jpg,1,468,831,475,836,344,3.162278
6,A20X_7_1160.jpg,1,1045,1335,1051,1339,418,4.000000
7,A20X_7_1161.jpg,1,1482,229,1489,234,438,4.472136
8,A20X_7_1161.jpg,1,1342,939,1345,944,548,2.000000
9,A20X_7_1161.jpg,1,934,971,940,975,557,2.000000


In [167]:
pore_data.iloc[21]

img      A20X_7_0266.jpg
label                  0
x1                  1206
y1                   766
x2                  1208
y2                   770
Name: 21, dtype: object

In [164]:
img_dir_paths, img_names = load_all_image_path("Corrections/4 - Object detection/porosity_data/images/test/")
# print(img_dir_paths[0])
# print(img_names[0])

for img, img_Name in zip(img_dir_paths, img_names):
    label_img, label_count = CCA_pore(img)
    print(str(img_Name) + ": " + str(label_count))
    #print(label_count)
    #plt.imshow(label_img)
    #plt.show()
    
    # append to data.txt
    # with open('img_Name.txt', 'a') as f:
    #    f.write(text)
            
    
    ## save image
    img_name = "Corrections/4 - Object detection/Labeled_CCA/Test/" + img_Name
    matplotlib.image.imsave(img_name, label_img)

A20X_7_0266.jpg: 50
A20X_7_0267.jpg: 50
A20X_7_0268.jpg: 47
A20X_7_0269.jpg: 45
A20X_7_0270.jpg: 47
A20X_7_1160.jpg: 199
A20X_7_1161.jpg: 188


### Training Images

In [19]:
img_dir_paths, img_names = load_all_image_path("Corrections/4 - Object detection/porosity_data/images/train/")
# print(img_dir_paths[0])
# print(img_names[0])

for img, img_Name in zip(img_dir_paths, img_names):
    label_img, label_count = CCA_pore(img)
    print(str(img_Name) + ": " + str(label_count))
    #plt.imshow(label_img)
    #plt.show()
    
    ## save image
    img_name = "Corrections/4 - Object detection/Labeled_CCA/Train/" + img_Name
    matplotlib.image.imsave(img_name, label_img)

NameError: name 'sqrt' is not defined

Width: 7 Height: 11 Area: 42
Width: 6 Height: 6 Area: 14
Width: 5 Height: 5 Area: 9
Width: 3 Height: 5 Area: 6
Width: 6 Height: 6 Area: 15
Width: 4 Height: 6 Area: 14
Width: 5 Height: 3 Area: 10
Width: 4 Height: 3 Area: 7
Width: 11 Height: 9 Area: 48
Width: 8 Height: 6 Area: 21
Width: 25 Height: 19 Area: 161
Width: 4 Height: 5 Area: 8
Width: 6 Height: 7 Area: 22
Width: 6 Height: 6 Area: 13
Width: 3 Height: 7 Area: 9
Width: 3 Height: 3 Area: 6
Width: 6 Height: 7 Area: 12
Width: 3 Height: 4 Area: 6
Width: 12 Height: 17 Area: 89
Width: 4 Height: 3 Area: 7
Width: 3 Height: 3 Area: 7
Width: 4 Height: 3 Area: 7
Width: 5 Height: 4 Area: 11
Width: 3 Height: 5 Area: 10
Width: 6 Height: 7 Area: 26
Width: 4 Height: 8 Area: 16
Width: 8 Height: 7 Area: 27
Width: 17 Height: 13 Area: 117
Width: 2 Height: 5 Area: 7
Width: 3 Height: 3 Area: 7
Width: 9 Height: 7 Area: 29
Width: 4 Height: 3 Area: 10
Width: 9 Height: 14 Area: 54
Width: 5 Height: 4 Area: 11
Width: 3 Height: 4 Area: 6
Width: 16 Height: 23 A

Width: 7 Height: 7 Area: 26
Width: 5 Height: 7 Area: 20
Width: 4 Height: 3 Area: 8
Width: 6 Height: 4 Area: 13
Width: 4 Height: 2 Area: 6
Width: 7 Height: 6 Area: 22
Width: 7 Height: 12 Area: 34
Width: 5 Height: 5 Area: 10
Width: 5 Height: 6 Area: 15
Width: 6 Height: 9 Area: 28
Width: 4 Height: 3 Area: 8
Width: 18 Height: 12 Area: 98
Width: 7 Height: 3 Area: 9
Width: 4 Height: 4 Area: 9
Width: 4 Height: 4 Area: 8
Width: 8 Height: 7 Area: 32
Width: 15 Height: 15 Area: 105
Width: 3 Height: 3 Area: 7
Width: 12 Height: 23 Area: 133
Width: 5 Height: 3 Area: 9
Width: 5 Height: 5 Area: 17
Width: 5 Height: 4 Area: 7
Width: 16 Height: 22 Area: 167
Width: 7 Height: 6 Area: 23
Width: 7 Height: 14 Area: 41
Width: 3 Height: 4 Area: 10
Width: 18 Height: 19 Area: 117
Width: 7 Height: 6 Area: 14
Width: 5 Height: 3 Area: 8
Width: 8 Height: 6 Area: 18
Width: 5 Height: 3 Area: 9
Width: 9 Height: 10 Area: 36
Width: 9 Height: 7 Area: 30
Width: 5 Height: 6 Area: 17
Width: 5 Height: 2 Area: 7
Width: 15 Heigh

Width: 9 Height: 6 Area: 29
Width: 6 Height: 5 Area: 16
Width: 5 Height: 5 Area: 14
Width: 5 Height: 8 Area: 26
Width: 6 Height: 5 Area: 14
Width: 3 Height: 4 Area: 8
Width: 7 Height: 5 Area: 19
Width: 5 Height: 6 Area: 13
Width: 8 Height: 8 Area: 40
Width: 3 Height: 5 Area: 7
Width: 6 Height: 10 Area: 29
Width: 4 Height: 4 Area: 7
Width: 5 Height: 4 Area: 7
Width: 17 Height: 10 Area: 71
Width: 5 Height: 5 Area: 13
Width: 8 Height: 7 Area: 34
Width: 14 Height: 13 Area: 70
Width: 5 Height: 4 Area: 10
Width: 4 Height: 3 Area: 7
Width: 15 Height: 22 Area: 143
Width: 4 Height: 4 Area: 10
Width: 4 Height: 4 Area: 9
Width: 15 Height: 22 Area: 168
Width: 4 Height: 2 Area: 7
Width: 2 Height: 4 Area: 6
Width: 10 Height: 9 Area: 43
Width: 4 Height: 5 Area: 10
Width: 9 Height: 5 Area: 14
Width: 7 Height: 10 Area: 23
Width: 5 Height: 6 Area: 10
Width: 15 Height: 12 Area: 78
Width: 3 Height: 4 Area: 7
Width: 4 Height: 3 Area: 7
Width: 4 Height: 4 Area: 9
Width: 18 Height: 15 Area: 143
Width: 5 Heig

Width: 9 Height: 10 Area: 33
Width: 3 Height: 4 Area: 7
Width: 9 Height: 10 Area: 44
Width: 5 Height: 4 Area: 11
Width: 3 Height: 3 Area: 6
Width: 12 Height: 12 Area: 68
Width: 5 Height: 7 Area: 18
Width: 12 Height: 9 Area: 57
Width: 10 Height: 8 Area: 45
Width: 8 Height: 6 Area: 18
Width: 14 Height: 8 Area: 47
Width: 16 Height: 11 Area: 60
Width: 3 Height: 3 Area: 7
Width: 2 Height: 5 Area: 6
Width: 6 Height: 9 Area: 23
Width: 13 Height: 13 Area: 94
Width: 9 Height: 11 Area: 66
Width: 7 Height: 6 Area: 25
Width: 6 Height: 8 Area: 29
Width: 3 Height: 4 Area: 9
Width: 3 Height: 3 Area: 7
Width: 3 Height: 3 Area: 6
Width: 5 Height: 3 Area: 8
Width: 4 Height: 7 Area: 11
Width: 4 Height: 5 Area: 11
Width: 15 Height: 13 Area: 65
Width: 15 Height: 17 Area: 127
Width: 4 Height: 4 Area: 8
Width: 13 Height: 20 Area: 111
Width: 5 Height: 5 Area: 14
Width: 4 Height: 4 Area: 9
Width: 6 Height: 7 Area: 18
Width: 5 Height: 5 Area: 12
Width: 4 Height: 3 Area: 6
Width: 3 Height: 5 Area: 9
Width: 9 Hei

...

Width: 4 Height: 4 Area: 12
Width: 4 Height: 6 Area: 9
Width: 8 Height: 7 Area: 29
Width: 3 Height: 3 Area: 7
Width: 10 Height: 11 Area: 81
Width: 4 Height: 4 Area: 8
Width: 7 Height: 7 Area: 26
Width: 13 Height: 14 Area: 63
Width: 12 Height: 17 Area: 126
Width: 4 Height: 5 Area: 10
Width: 10 Height: 12 Area: 47
Width: 4 Height: 6 Area: 10
Width: 5 Height: 5 Area: 12
Width: 2 Height: 4 Area: 6
Width: 11 Height: 11 Area: 42
Width: 10 Height: 13 Area: 82
Width: 7 Height: 8 Area: 31
Width: 14 Height: 18 Area: 125
Width: 7 Height: 6 Area: 29
Width: 7 Height: 7 Area: 25
Width: 9 Height: 13 Area: 34
Width: 13 Height: 9 Area: 50
Width: 10 Height: 10 Area: 43
Width: 13 Height: 17 Area: 121
Width: 10 Height: 6 Area: 24
Width: 6 Height: 3 Area: 13
Width: 6 Height: 9 Area: 36
Width: 10 Height: 11 Area: 71
Width: 6 Height: 6 Area: 12
Width: 4 Height: 5 Area: 11
Width: 5 Height: 3 Area: 10
Width: 16 Height: 15 Area: 103
Width: 2 Height: 4 Area: 6
Width: 4 Height: 5 Area: 9
Width: 5 Height: 4 Area: 

Width: 12 Height: 10 Area: 82
Width: 4 Height: 3 Area: 7
Width: 15 Height: 20 Area: 144
Width: 5 Height: 9 Area: 28
Width: 5 Height: 7 Area: 20
Width: 5 Height: 7 Area: 22
Width: 5 Height: 4 Area: 13
Width: 6 Height: 5 Area: 11
Width: 3 Height: 4 Area: 8
Width: 11 Height: 13 Area: 79
Width: 7 Height: 9 Area: 32
Width: 6 Height: 4 Area: 14
Width: 9 Height: 8 Area: 42
Width: 3 Height: 4 Area: 6
Width: 9 Height: 7 Area: 36
Width: 4 Height: 3 Area: 7
Width: 4 Height: 5 Area: 10
Width: 12 Height: 6 Area: 31
Width: 5 Height: 5 Area: 14
Width: 13 Height: 10 Area: 51
Width: 3 Height: 2 Area: 6
Width: 6 Height: 8 Area: 24
Width: 4 Height: 5 Area: 11
Width: 4 Height: 6 Area: 15
Width: 4 Height: 3 Area: 7
Width: 9 Height: 12 Area: 46
Width: 5 Height: 4 Area: 9
Width: 7 Height: 8 Area: 27
Width: 5 Height: 2 Area: 7
Width: 9 Height: 8 Area: 27
Width: 6 Height: 6 Area: 15
Width: 6 Height: 5 Area: 12
Width: 10 Height: 14 Area: 45
Width: 11 Height: 7 Area: 34
Width: 10 Height: 20 Area: 63
Width: 4 Hei

Width: 8 Height: 10 Area: 36
Width: 6 Height: 9 Area: 33
Width: 4 Height: 3 Area: 8
Width: 11 Height: 7 Area: 35
Width: 4 Height: 4 Area: 10
Width: 4 Height: 3 Area: 10
Width: 14 Height: 19 Area: 170
Width: 4 Height: 5 Area: 12
Width: 4 Height: 12 Area: 25
Width: 7 Height: 11 Area: 34
Width: 5 Height: 10 Area: 20
Width: 6 Height: 6 Area: 13
Width: 3 Height: 3 Area: 6
Width: 4 Height: 2 Area: 7
Width: 10 Height: 14 Area: 80
Width: 6 Height: 7 Area: 22
Width: 6 Height: 4 Area: 15
Width: 14 Height: 8 Area: 58
Width: 10 Height: 8 Area: 33
Width: 3 Height: 5 Area: 6
Width: 2 Height: 3 Area: 6
Width: 4 Height: 3 Area: 7
Width: 4 Height: 7 Area: 13
Width: 5 Height: 3 Area: 7
Width: 3 Height: 3 Area: 7
Width: 9 Height: 6 Area: 19
Width: 4 Height: 4 Area: 8
Width: 7 Height: 4 Area: 14
Width: 4 Height: 7 Area: 14
Width: 5 Height: 2 Area: 6
Width: 5 Height: 3 Area: 7
Width: 3 Height: 4 Area: 6
Width: 3 Height: 4 Area: 10
Width: 3 Height: 4 Area: 7
Width: 5 Height: 4 Area: 11
Width: 8 Height: 6 Ar

Width: 5 Height: 6 Area: 16
Width: 6 Height: 6 Area: 19
Width: 4 Height: 2 Area: 6
Width: 14 Height: 18 Area: 118
Width: 12 Height: 8 Area: 32
Width: 4 Height: 4 Area: 9
Width: 5 Height: 5 Area: 12
Width: 5 Height: 8 Area: 19
Width: 7 Height: 13 Area: 45
Width: 4 Height: 5 Area: 9
Width: 8 Height: 6 Area: 23
Width: 2 Height: 4 Area: 7
Width: 4 Height: 5 Area: 13
Width: 7 Height: 13 Area: 47
Width: 8 Height: 9 Area: 25
Width: 5 Height: 4 Area: 14
Width: 15 Height: 9 Area: 69
Width: 5 Height: 6 Area: 12
Width: 9 Height: 9 Area: 37
Width: 4 Height: 5 Area: 9
Width: 2 Height: 6 Area: 9
Width: 2 Height: 4 Area: 6
Width: 5 Height: 9 Area: 18
Width: 6 Height: 3 Area: 7
Width: 5 Height: 7 Area: 20
Width: 4 Height: 3 Area: 7
Width: 6 Height: 4 Area: 11
Width: 4 Height: 4 Area: 7
Width: 4 Height: 2 Area: 6
Width: 6 Height: 5 Area: 13
Width: 7 Height: 5 Area: 16
Width: 14 Height: 7 Area: 54
Width: 11 Height: 8 Area: 40
Width: 5 Height: 6 Area: 13
Width: 7 Height: 12 Area: 26
Width: 9 Height: 6 Ar

Width: 3 Height: 3 Area: 6
Width: 5 Height: 5 Area: 13
Width: 4 Height: 5 Area: 13
Width: 6 Height: 7 Area: 21
Width: 12 Height: 9 Area: 65
Width: 11 Height: 8 Area: 44
Width: 6 Height: 5 Area: 11
Width: 4 Height: 4 Area: 9
Width: 5 Height: 4 Area: 9
Width: 4 Height: 8 Area: 14
Width: 5 Height: 5 Area: 18
Width: 6 Height: 6 Area: 16
Width: 4 Height: 3 Area: 7
Width: 7 Height: 6 Area: 18
Width: 5 Height: 7 Area: 18
Width: 9 Height: 10 Area: 36
Width: 8 Height: 7 Area: 31
Width: 3 Height: 4 Area: 6
Width: 7 Height: 6 Area: 18
Width: 4 Height: 6 Area: 8
Width: 6 Height: 8 Area: 17
Width: 4 Height: 3 Area: 6
Width: 3 Height: 4 Area: 8
Width: 5 Height: 3 Area: 9
Width: 8 Height: 8 Area: 27
Width: 4 Height: 3 Area: 7
Width: 8 Height: 7 Area: 26
Width: 3 Height: 4 Area: 6
Width: 4 Height: 3 Area: 7
Width: 8 Height: 4 Area: 17
Width: 11 Height: 10 Area: 43
Width: 10 Height: 11 Area: 40
Width: 2 Height: 4 Area: 6
Width: 7 Height: 8 Area: 31
Width: 3 Height: 6 Area: 10
Width: 4 Height: 3 Area: 7

### Validation Images

In [ ]:
img_dir_paths, img_names = load_all_image_path("Corrections/4 - Object detection/porosity_data/images/val/")
print(img_dir_paths[0])
print(img_names[0])

for img, img_Name in zip(img_dir_paths, img_names):
    label_img, label_count = CCA_pore(img)
    print(str(img_Name) + ": " + str(label_count))
    #plt.imshow(label_img)
    #plt.show()
    
    ## save image
    img_name = "Corrections/4 - Object detection/Labeled_CCA/Valid/" + img_Name
    matplotlib.image.imsave(img_name, label_img)

Width: 3 Height: 4 Area: 6
Width: 3 Height: 3 Area: 6
Width: 9 Height: 9 Area: 35
Width: 12 Height: 10 Area: 47
Width: 10 Height: 8 Area: 23
Width: 8 Height: 4 Area: 16
Width: 5 Height: 5 Area: 12
Width: 8 Height: 6 Area: 20
Width: 7 Height: 7 Area: 21
Width: 8 Height: 8 Area: 33
Width: 4 Height: 4 Area: 10
Width: 6 Height: 4 Area: 8
Width: 6 Height: 2 Area: 7
Width: 4 Height: 3 Area: 6
Width: 11 Height: 11 Area: 57
Width: 7 Height: 10 Area: 28
Width: 4 Height: 3 Area: 6
Width: 5 Height: 3 Area: 6
Width: 5 Height: 6 Area: 17
Width: 3 Height: 4 Area: 6
Width: 8 Height: 7 Area: 27
Width: 9 Height: 12 Area: 43
Width: 6 Height: 6 Area: 17
Width: 6 Height: 4 Area: 15
Width: 3 Height: 4 Area: 8
Width: 5 Height: 6 Area: 12
Width: 7 Height: 10 Area: 30
Width: 5 Height: 6 Area: 15
Width: 7 Height: 8 Area: 25
Width: 4 Height: 5 Area: 9
Width: 14 Height: 8 Area: 64
Width: 10 Height: 10 Area: 39
Width: 3 Height: 6 Area: 13
Width: 4 Height: 4 Area: 8
Width: 10 Height: 10 Area: 28
Width: 4 Height: 6

Width: 4 Height: 2 Area: 6
Width: 14 Height: 10 Area: 68
Width: 3 Height: 5 Area: 8
Width: 14 Height: 19 Area: 91
Width: 4 Height: 3 Area: 7
Width: 5 Height: 6 Area: 15
Width: 15 Height: 10 Area: 103
Width: 6 Height: 6 Area: 15
Width: 14 Height: 13 Area: 62
Width: 7 Height: 7 Area: 25
Width: 6 Height: 4 Area: 10
Width: 5 Height: 4 Area: 12
Width: 6 Height: 4 Area: 11
Width: 12 Height: 9 Area: 39
Width: 5 Height: 4 Area: 12
Width: 5 Height: 4 Area: 7
Width: 9 Height: 7 Area: 32
Width: 3 Height: 5 Area: 11
Width: 9 Height: 9 Area: 41
Width: 4 Height: 3 Area: 7
Width: 6 Height: 6 Area: 16
Width: 9 Height: 10 Area: 42
Width: 4 Height: 3 Area: 9
Width: 7 Height: 9 Area: 39
Width: 10 Height: 8 Area: 39
Width: 10 Height: 12 Area: 62
Width: 3 Height: 5 Area: 7
Width: 8 Height: 7 Area: 27
Width: 5 Height: 10 Area: 21
Width: 4 Height: 4 Area: 9
Width: 12 Height: 7 Area: 40
Width: 9 Height: 7 Area: 42
Width: 3 Height: 3 Area: 6
Width: 3 Height: 3 Area: 8
Width: 7 Height: 9 Area: 43
Width: 3 Heigh

Width: 3 Height: 3 Area: 7
Width: 5 Height: 4 Area: 11
Width: 7 Height: 9 Area: 44
Width: 7 Height: 8 Area: 27
Width: 14 Height: 8 Area: 70
Width: 3 Height: 3 Area: 6
Width: 6 Height: 10 Area: 31
Width: 6 Height: 6 Area: 22
Width: 6 Height: 6 Area: 14
Width: 13 Height: 15 Area: 113
Width: 3 Height: 5 Area: 9
Width: 11 Height: 14 Area: 91
Width: 10 Height: 11 Area: 61
Width: 11 Height: 9 Area: 43
Width: 16 Height: 24 Area: 143
Width: 10 Height: 10 Area: 43
Width: 2 Height: 6 Area: 9
Width: 15 Height: 14 Area: 114
Width: 9 Height: 4 Area: 25
Width: 4 Height: 3 Area: 7
Width: 13 Height: 14 Area: 74
Width: 6 Height: 6 Area: 13
Width: 7 Height: 8 Area: 25
Width: 9 Height: 13 Area: 47
Width: 3 Height: 5 Area: 6
Width: 6 Height: 7 Area: 17
Width: 11 Height: 11 Area: 58
Width: 9 Height: 12 Area: 66
Width: 8 Height: 7 Area: 34
Width: 16 Height: 14 Area: 98
Width: 3 Height: 3 Area: 6
Width: 4 Height: 4 Area: 7
Width: 7 Height: 3 Area: 11
Width: 26 Height: 20 Area: 192
Width: 8 Height: 5 Area: 27

Width: 4 Height: 6 Area: 12
Width: 11 Height: 13 Area: 81
Width: 4 Height: 2 Area: 6
Width: 13 Height: 11 Area: 61
Width: 4 Height: 4 Area: 9
Width: 8 Height: 14 Area: 65
Width: 3 Height: 7 Area: 12
Width: 4 Height: 2 Area: 7
Width: 4 Height: 3 Area: 6
Width: 12 Height: 7 Area: 42
Width: 5 Height: 5 Area: 13
Width: 6 Height: 6 Area: 21
Width: 3 Height: 2 Area: 6
Width: 10 Height: 14 Area: 34
Width: 3 Height: 3 Area: 7
Width: 8 Height: 7 Area: 27
Width: 4 Height: 4 Area: 9
Width: 15 Height: 8 Area: 59
Width: 5 Height: 8 Area: 20
Width: 4 Height: 6 Area: 16
Width: 5 Height: 4 Area: 8
Width: 5 Height: 5 Area: 11
Width: 4 Height: 4 Area: 8
Width: 5 Height: 4 Area: 10
Width: 5 Height: 5 Area: 13
Width: 6 Height: 5 Area: 17
Width: 6 Height: 7 Area: 21
Width: 3 Height: 5 Area: 7
Width: 4 Height: 4 Area: 7
Width: 7 Height: 9 Area: 22
Width: 2 Height: 5 Area: 6
Width: 4 Height: 7 Area: 11
Width: 10 Height: 13 Area: 44
Width: 10 Height: 7 Area: 42
Width: 6 Height: 6 Area: 19
Width: 4 Height: 4 A

In [ ]:
#Source
# 1 - https://pyimagesearch.com/2021/02/22/opencv-connected-component-labeling-and-analysis/
# 2 - https://datacarpentry.org/image-processing/08-connected-components/index.html